
# Projekt - Etap 2 - IUM 2021L
*// Konrad Bratosiewicz // Mateusz Chruściel //* 

# Budowa modelu
## Dobór danych uczących
Po wcześniejszej analizie danych do naszego modelu postanowiliśmy użyć:
- liczby wizyt produktu
- czas oglądania produktu w sekundach (znormalizowany metodą min-max)
- liczba wejść na minutę
- maksymalną zaoferowaną do tej pory zniżkę
- aktualnie oferowaną zniżkę
- czy klient zakupił finalnie przedmiot (-1 = nie, 1 = tak)

## Założenia do wyników modelu
Jak usaliliśmy podczas analizy danych żeby nasz model dawał sensowne wyniki musi przewidywać czy klient zakupi przedmiot z prawdopodobieństwem większym niż 57%, bo tyle wynosi prawdopodobieństwo wyboru klasy większościowej nie_kupił.

## 1 Model zachłanny
Pierwszy model daje zawszę zniżkę 20% bo gwarantuje ona przekonanie jak największej liczby osób

## 2 Wyuczony model
Na podstawie danych wejściowych uczymy model przewidywać czy dana "produkto-sesja" zakończy się zakupem. Model zwraca nam liczbę z przedziału <-1,1>. 

Dla takiego modelu by uzyskać predykcję zniżki wysyłamy zapytanie zawierające wszystkie dane ze wszystkimi możliwościami zniżki. Na przykład dla danych:
```
{"d_time":0.100068132,"n_visits":2,"visits_per_minute":0.0001321401,"max_disc":0,"curr_disc":0,"is_purchased":-1}
```
Zapytanie będzie wyglądało następująco:
```
[[0.100068132, 2, 0.0001321401, 0, 0],[0.100068132, 2, 0.0001321401, 0, 5],[0.100068132, 2, 0.0001321401, 0, 10],[0.100068132, 2, 0.0001321401, 0, 15],[0.100068132, 2, 0.0001321401, 0, 20]]
```
Przykładowa odpowiedź:
```
[-0.5271583795547485, 0.9961207509040833, 0.9995086789131165, 0.9996641874313354, 0.9995707273483276]
```

Model zwraca nam wtedy listę 5 liczb odpowiadającym według niego prawdopodobieństwu kupienia przedmiotu dla kolejnych zniżek. Finalna zniżka może być wybrana na wiele sposobów, my przygotowaliśmy 3 do porównania. 

Dokładna architektura oraz funkcje modelu w pliku model.py
 

In [18]:
import pandas as pd 
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt 

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from tqdm.notebook import tqdm
from random import randint

from model import Model
from load_data import load_data

Załadowanie danych

In [19]:
train_dataLoader, test_dataLoader, = load_data("data3/normal_vpm.json",batch_size=128,test_size=0.1)

Stworzenie modelu

In [20]:
model1 = Model()
optimizer = optim.Adam(model1.parameters(),lr=0.001)
criterion = nn.MSELoss()

Trenowanie modelu

In [21]:
model1.fit(train_dataLoader,optimizer,criterion,epochs=50)

Losses:   0%|          | 0/50 [00:00<?, ?it/s]

Testowanie modelu - zakładamy że model miał rację jeśli wartość absolutna wyniku jest większa niż próg

In [22]:
model1.test(test_dataLoader,treshold=0.5)


Corectly predicted: 1813 out of 2034
Test accuracy: 89.13%


Otrzymaliśmy dokładność na poziomie powyżej 85%, więc możemy spokojnie założyć że nasz model rzeczywiście nauczył się rozpoznawać czy sesja zakończy się zakupem.

## Opcje wyboru zniżki na podstawie wyników modelu

### Opcja 1: Zniżka dla maksymalnego prawdopodobieństwa

In [23]:
def best_discount1(discount_list):
    return discount_list.index(max(discount_list)) * 5


### Opcja 2: Zniżka, dla której prawdopodobieństwo zakupu jest większe niż określony próg

In [24]:
def best_discount2(discount_list,threshold):
    for i,disc in enumerate(discount_list):
        if disc > threshold:
            return i * 5
    return i * 5

### Opcja 3: Losowa zniżka dla której klient ma większe szanse, że kupi niż nie kupi

In [25]:
def best_discount3(discount_list):
    if max(discount_list) <= 0:
        return 20
    while True:
        idx = randint(0,4)
        if discount_list[idx] > 0:
            return idx * 5
        

Pobranie wyników predykcji dla zniżek na podstawie danych testujących 

In [26]:
def get_discounts(dataloader):
    out = []
    bought = 0
    for X,Y in dataloader:
        for y in Y:
            if y > 0:
                bought +=1
        for x in X:
            x_ = x.squeeze().tolist()[:4]
            inp = torch.Tensor([x_ + [d*5] for d in range(5)])
            out.append(model1(inp).squeeze().tolist())
    return out, bought

## Badanie średnich zniżek dla różnych opcji

In [28]:
list1 = []
list2 = []
list3 = []
disc_list, bought = get_discounts(test_dataLoader)
for disc in disc_list:
    list1.append(best_discount1(disc))
    list2.append(best_discount2(disc,0.8))
    list3.append(best_discount3(disc))
print(f'BOUGHT: {bought} out of {len(test_dataLoader.dataset)}')
print(f'Option 1: {np.average(list1):.5}%')
print(f'Option 2: {np.average(list2):.5}%')
print(f'Option 2: {np.average(list3):.5}%')


BOUGHT: 865 out of 2034
Option 1: 18.68%
Option 2: 13.606%
Option 2: 15.691%


Jak widać po powyższych wynikach niezależnie od wybranej opcji doboru zniżki zawsze otrzymujemy średnią zniżkę niższą niż 20%.

Najlepsze wyniki ok 13.6% otrzymaliśmy dla opcji 2 tj. najniższa zniżka, dla której prawdopodobieństwo kupienia będzie wyższe niż zadany próg, w tym przypadku 0.8.

Dzięki zastosowaniu naszego modelu zapewniamy te samo prawdopodobieństwo zakupu co przy dawaniu zawsze maksymalnej zniżki, redukując ją przy tym o dobre kilkadziesiąt procent. Oczywiście nie jesteśmy w stanie przeprowadzić testów w prawdziwym sklepie, więc na suchych danych nasz możliwości są dość ohraniczone, jednak wydaje nam się, że jest to wystarczający dowód działania naszego modelu.